In [1]:
import torch
from meshplot import plot
import sys 
sys.path.append('../')
import radfoam
from radfoam_model.mesh_utils import marching_tetrahedra
device='cuda'

In [2]:
num_init_points = 30000
primal_points = (torch.rand(num_init_points, 3, device='cuda')-.5)*2.

primal_values = torch.norm(primal_points,dim=1)-.5
primal_features = primal_points[:, 0]

triangulation = radfoam.Triangulation(primal_points)
perm = triangulation.permutation().to(torch.long)
primal_points = primal_points[perm]
primal_values = primal_values[perm]
primal_features = primal_features[perm]

In [8]:
primal_features.shape

torch.Size([30000])

In [9]:
primal_values[(0<primal_values)*(primal_values<.001)]=0

v, f, feat = marching_tetrahedra(triangulation.tets().long(), primal_values, primal_points, primal_features[:, None])

import igl
import numpy as np
# Convert tensors to numpy arrays
v_np = v.cpu().detach().numpy()
f_np = f.cpu().detach().numpy()

face_normals = igl.per_face_normals(v_np, f_np, np.array([1.0, 1.0, 1.0], dtype=np.float32))

plot(v.cpu().detach().numpy(), f.cpu().detach().numpy(), feat.cpu().detach().numpy())
# plot(v_np, f_np, (face_normals+1)/2.)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000196…

In [13]:
def export_obj(nv, nf, name: str, nvn=None):
    if name[-4:] != ".obj":
        name += ".obj"
    try:
        file = open(name, "x")
    except:
        file = open(name, "w")
    # file.write("o {} \n".format(name))

    for v in nv:
        file.write("v {} {} {}\n".format(*v))
    file.write("\n")

    if nvn is not None:
        for vn in nvn:
            file.write("vn {} {} {}\n".format(*vn))
    file.write("\n")

    for face in nf:
        file.write("f " + " ".join([str(fi + 1) for fi in face]) + "\n")
    file.write("\n")

In [14]:
export_obj(v.cpu().detach().numpy(), f.cpu().detach().numpy(), 'test.obj')

### New Method


In [4]:
len(v)

19668

In [5]:
v, f, feat, edge = marching_tetrahedra(triangulation.tets().long(), primal_values, primal_points, primal_features, True)
